In [ ]:
import os
import torch
import torchvision
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np
import csv
import pandas as pd
import seaborn as sns
#import matplotlib.pylab as plt


from kiwissenbase.data import dataloaders
from kiwissenbase.data.datasets import CaltechPedestrian
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np

## create data loader

In [ ]:
data_loader_args = {"root_dir": "/data/anna/data/caltech_dataset/",
                    "batch_size": 8,
                    "validation_batch_size": 4,
                    "num_workers": 0,
                    "pin_memory": True,
                    #"collate_fn": collate_fn,
                    "normal_mean": (0.5, 0.5, 0.5),
                    "normal_std": (0.5, 0.5, 0.5),
                    "different_size_target": True,
                    "subset": "annotated-pedestrians",                    
                    "target_transform":{"module": "kiwissenbase.models.object_detection",
                                        "class_name": "FasterRCNN",
                                        "method_name": "target_transform"}}

In [ ]:
loader = dataloaders.CaltechPedastrianDataLoader(device="cpu", **data_loader_args)

In [ ]:
loader.train.dataset

In [ ]:
loader.test.dataset

## load and test saved model

In [ ]:
checkpoint = torch.load("/data/anna/pedestrian_detection_models/faster_rcnn_tuned_Caltech.pth")

In [ ]:
checkpoint

In [ ]:
## load model architecture and adapt output
import torchvision 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

In [ ]:
model.load_state_dict(checkpoint['model_state'])

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.eval()
model.to(device)

## register forward hooks on the dissected layer

In [ ]:
def get_activation(name):
    def hook(model, input, output):
        unit_activations[name] = output.detach()
    return hook

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

In [ ]:
unit_activations = {}
conv_unit = "backbone.body.layer4[1].conv2"
eval(f"model.{conv_unit}.register_forward_hook")(get_activation(conv_unit))

### test model and data

In [ ]:
image,targets = next(iter(loader.test.dataset))
image.to(device)
predictions = model([image.to(device)]) 

In [ ]:
predictions

In [ ]:
unit_activations

## get unit activation (max and average activation for each image) for the test set

In [ ]:
unit_activations = {}
conv_unit = "backbone.body.layer4[1].conv2"
eval(f"model.{conv_unit}.register_forward_hook")(get_activation(conv_unit))

In [ ]:
from tqdm import tqdm

In [ ]:
max_activations = []
avg_activations = []
pedestrian_image = []
detected_pedestrian_image = []
all_targets = []
all_predictions = []
with torch.no_grad():
    for local_batch in tqdm(loader.test):
        pedestrian_image.extend([any(item["labels"]==1) for item in local_batch[1]])
        local_batch_images = torch.stack(local_batch[0]).to(device)
        all_targets.extend(local_batch[1])
        output = model(local_batch_images)
        output_cpu = [{"boxes": item["boxes"].to("cpu"), "labels": item["labels"].to("cpu"), "scores": item['scores'].to("cpu")}for item in output]
        all_predictions.extend(output_cpu)
        detected_pedestrian_image.extend([any(item["labels"]==1) for item in output])
        for activation in unit_activations[conv_unit]:
            flat_tensor = torch.flatten(activation, start_dim=1)
            max_activations.append(torch.max(flat_tensor, dim=1)[0])
            avg_activations.append(torch.mean(flat_tensor, dim=1))

In [ ]:
print("total images", len(pedestrian_image))
print("images with at least one labelled pedestrian", sum(pedestrian_image))

In [ ]:
print("images with at least one detected pedestrian", sum(detected_pedestrian_image))

In [ ]:
max_activations_tensor = torch.stack(max_activations)
avg_activations_tensor = torch.stack(avg_activations)

In [ ]:
avg_activations_numpy = avg_activations_tensor.detach().to("cpu").numpy()
max_activations_numpy = max_activations_tensor.detach().to("cpu").numpy()

In [ ]:
unit_activations[conv_unit].shape

## Read in the netdissect result

In [ ]:
path_to_result = "/data/anna/results/pytorch_fasterrcnn_resnet50_fpn_caltech_backbone.body.layer4[1].conv2/tally.csv"

In [ ]:
data = [line for line in csv.reader(open(path_to_result))]
data_all = [{"unit": int(item[0])-1, "concept":item[2], "score":item[3]} for item in data[1:]]
data_top = data_all[:30]
print("Top scoring units\n")
pd.DataFrame(data_top)

### find the relevant detectors from netdissect results

In [ ]:
print("all concepts in layer")
print(set([item[2] for item in data[1:]]))

In [ ]:
relevant_concepts = ["head", "hair", "arm", "wheel", "car", "sidewalk","road","neck","mouth","person","leg","back","foot"]

In [ ]:
data_top_relevant = [{"unit": int(item[0])-1, "concept":item[2], "score":item[3]} for item in data[1:] if item[2] in relevant_concepts][:30]
print("Top scoring relevant units\n")
pd.DataFrame(data_top_relevant)

## check correlation of detector units

### select metric

In [ ]:
metric = avg_activations_numpy
#metric = max_activations_numpy

### select units to compare

In [ ]:
unit1 = 97
unit2 = 510

In [ ]:
x = metric[:,unit1]
y = metric[:,unit2]

In [ ]:
unit1_display = [f"{item[0]}_{item[2]}_{round(float(item[3]),3)}" for item in data[1:] if int(item[0])==unit1+1][0]
unit2_display = [f"{item[0]}_{item[2]}_{round(float(item[3]),3)}" for item in data[1:] if int(item[0])==unit2+1][0]

In [ ]:
plt.scatter(x, y)
plt.xlabel(f"unit {unit1_display}")
plt.ylabel(f"unit {unit2_display}")
plt.title(f"correllation {round(pearsonr(x,y)[0],3)}")
plt.show()

### calculate correllation heatmap

In [ ]:
#selected_data = data_top
selected_data = data_top_relevant

In [ ]:
cross_correllation = np.zeros((len(selected_data),len(selected_data)))
axes_labels = [f"{item['unit']}_{item['concept']}_{round(float(item['score']),3)}" for item in selected_data]

In [ ]:
for unit_ind1,unit1 in tqdm(enumerate(selected_data)):
    for unit_ind2,unit2 in enumerate(selected_data[unit_ind1:]):
        x = metric[:,unit1["unit"]]
        y = metric[:,unit2["unit"]]
        r = pearsonr(x,y)[0]
        cross_correllation[unit_ind1,unit_ind1+unit_ind2] = cross_correllation[unit_ind1+unit_ind2,unit_ind1] = r

In [ ]:
sns.set(rc={'figure.figsize':(13,13)})

ax = sns.heatmap(cross_correllation, linewidth=0.5,cmap="seismic",vmin=-1,vmax=1)
ax.set_xticks(np.arange(0.5,30.5,1), axes_labels,rotation=90)
ax.set_yticks(np.arange(0.5,30.5,1), axes_labels, rotation=0)
#ax.set_xticklabels(axes_labels, rotation=45)
plt.show()

## examine single detectors with repsect to activation for pedestrian vs non-pedestrian images

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu

### examine metric for pdestrian vs non-pedestrian images (detected) - max seems to be working better

In [ ]:
#metric = avg_activations_numpy
metric = max_activations_numpy

In [ ]:
#selected_data = data_top
selected_data = data_top_relevant

In [ ]:
selected_data

In [ ]:
unit = 327
# max activation, 327

In [ ]:
pedestrian_activations = [activation[unit] for activation,ped_prsent in zip(metric,detected_pedestrian_image) if ped_prsent] 
non_pedestrian_activations = [activation[unit] for activation,ped_prsent in zip(metric,detected_pedestrian_image) if not ped_prsent] 

In [ ]:
len(pedestrian_activations),len(non_pedestrian_activations)

In [ ]:
print("mean/std metric in positive predictions(pedestrian detected)", np.mean(pedestrian_activations), np.std(pedestrian_activations))
print("mean/std metric in negative predictions(no pedestrian detected)", np.mean(non_pedestrian_activations), np.std(non_pedestrian_activations))

### visualise the two populations

In [ ]:
p_values = [ttest_ind(non_pedestrian_activations, pedestrian_activations)[1],
            mannwhitneyu(non_pedestrian_activations, pedestrian_activations)[1]]

In [ ]:
concept = [item['concept'] for item in selected_data if item['unit']==unit][0]
figure_title = f"unit: {unit} - concept: {concept} p_values: Ttest:{round(p_values[0],5)}, Mann-Whitney test:{round(p_values[1],5)}"

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(15,5)) # Instantiate figure and axes object
axes[0].hist(non_pedestrian_activations, label="nums1", histtype="step", density=True, linewidth=2) # Plot histogram of nums1
axes[0].hist(pedestrian_activations, label="nums2", histtype="step", density=True, linewidth=2) # Plot histogram of nums2
axes[0].legend(["no pedestrian detection", ">=1 pedestrian detection"])
axes[1].boxplot([non_pedestrian_activations,pedestrian_activations])
axes[1].set_xticklabels(["no pedestrian \ndetection", ">=1 pedestrian\n detection"])
fig.suptitle(figure_title,fontsize=15)
plt.show()

## todo how to isolate and examine detectors against individual pedestrian predections in an image

for an image with false classifications (fps/fns) project activation of unit back to input image

get activation around expected area of missed bounding box / area around prediction of false bounding box

## todo check behaviour for fp/tp/fn/tn

## todo check behaviour with regards to size of (visible) bounding box